## OWASP ZAP Mini Workshop - ZAPv2.7.0

#### Abhay Bhargav - we45

#### Objective
Over the last 2 years, I have worked with and implemented several SAST and DAST tools as part of a Continuous Delivery Pipeline. In this time, I have seen that there are few tools that match the flexibility and capabilities of OWASP's Zed Attack Proxy, especially from the perspective of integrating into a Continuous Delivery Pipeline. 

As part of training programs that I have delivered at OWASP AppSecUSA, EU and DEFCON, I have trained several folks on OWASP ZAP's API and Scripting features. This is a shorter version of the same training program that I have delivered in these places. 

The objective of this is as follows: 
- To get someone quickly up and running with ZAP and its feature-rich API and Scripting Features
- To NOT use presentations. It takes FOREVER prepping presentation. Writing everything in Markdown and Python is far easier
- To cover essential functionality that I think would be useful to folks. I am sure I will be missing some cool features here. Please bear with me
- To reference some other useful tools and libraries that I have been working with, either ZAP related or otherwise. 

#### Requirements
- Python 3.6.1
- Please install all requirements from `requirements.txt`. This includes ZAP's Python API
- Download and Install ZAP 2.7.0 (Download from https://github.com/zaproxy/zaproxy/wiki/Downloads)

#### Warnings
This code is given to you as is, where is. I don't take any responsibility for how/where you use it. This is NOT production ready in any way.

#### Thanks
- I would like to thank Simon Bennetts (@psiinon) and the rest of the OWASP ZAP Development Team. They do a truly admirable job of maintaining and managing ZAP for all of us. 
- I would like to thank a few members of my amazing team! - Nithin, Sharath and Tilak. Thanks for all that you do

#### Other Useful Projects
- ZAP Robot Framework Integration - https://bitbucket.org/we45abhay/robozap
- Vulnerable App used in this exercise - https://bitbucket.org/we45abhay/defcon_vul_app/

###### Let's get started.....


### Pre-start Setup Instructions

- Download and Install ZAP
- Install some plugins from the ZAP Marketplace, namely: 
    - Python Scripting
    - Export Report
- You may choose to install several other plugins. However, for this training, you only need these plugins
- Open ZAP > Tools > Options > API 
    - Make sure that it's enabled, for UI as well
    ![ZAP API Screen](api_zap.jpg)
    - I have disabled API key in this case. You can enable if you would like. However, remember that you need the API Key for every call to ZAP's API if you enable the API Key.
    

### ZAP 2.7.0 API

If you enable the UI Option in ZAP's API Options screen (as done above), you will have access to ZAP's API directly over the browser. You will typically have ZAP running on port 8080 of your localhost. In my case, I have it running on port 8090

The API is available only when you have started ZAP. 

![ZAP API Screen](api_screen.jpg)

Very soon, we are going to start zap using ZAP's CLI and get it up and running. This is very useful when you want to start ZAP without using the UI. 

In [9]:
# we will be using python's subprocess to start ZAP in GUI and headless modes

import subprocess
import os
from IPython.display import display

#GUI ZAP
base_path = '/Applications/OWASP_ZAP.app/Contents/Java/'
gui_command = base_path + 'zap.sh -config api.disablekey=true -port 8090'
# you can use the config param to specify set specific configurations you need when you launch the CLI.
# In this case, I am (actually don't need to) starting ZAP with the API Key disabled and listening port 8090

headless_command = base_path + 'zap.sh -daemon -config api.disablekey=true -port 8090'
#by specifying 'daemon' in the CLI, ZAP starts in Headless mode

zap_process = subprocess.Popen(gui_command.split(' '), stdout = open(os.devnull, 'w'))

If you have run the code above correctly, you should see that ZAP is now running in GUI Mode. We will be using GUI Mode for this workshop, so you can easily see everything going on within ZAP.

Now that we have ZAP open and running, let's use its API to perform some scans. I will be using it's Python API to perform this. However, ZAP can be used with Java, JavaScript, PHP, Ruby and directly as a REST API

##### Links: 
- NPM: https://www.npmjs.com/package/zaproxy
- Java: https://github.com/zaproxy/zaproxy/releases
- Ruby: https://github.com/vpereira/owasp_zap

Let's start with running the spider against intentionally vulnerable site `demo.testfire.net`

In [11]:
from zapv2 import ZAPv2 as ZAP #import ZAP library
import time

zap = ZAP(proxies = {'http': 'http://localhost:8090', 'https': 'http://localhost:8090'})
#setting the local ZAP instance that is open on your local system

target_site = 'http://demo.testfire.net'

zap.urlopen(target_site)
#opens up the the target site. Makes a single GET request

spider_id = zap.spider.scan(target_site)
#this line of code kicks off the ZAP Default Spider. This returns an ID value for the spider

print("Spider ID for the initiated spider scan is: {0}".format(spider_id))


#now we can start monitoring the spider's status
while int(zap.spider.status(spider_id)) < 100:
    print("Current Status of ZAP Spider: {0}%".format(zap.spider.status(spider_id)))
    time.sleep(4)

Spider ID for the initiated spider scan is: 1
Current Status of ZAP Spider: 0%
Current Status of ZAP Spider: 10%
Current Status of ZAP Spider: 10%
Current Status of ZAP Spider: 30%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 55%
Current Status of ZAP Spider: 55%
Current Status of ZAP Spider: 71%
Current Status of ZAP Spider: 83%
Current Status of ZAP Spider: 90%
Current Status of ZAP Spider: 92%
Current Status of ZAP Spider: 93%


If everything went well, you should see that the spider ran successfully, and identified a bunch of in-scope and Out of scope URLs and params. 

Before we proceed with scans and other "vulnerability discovery" activities. Let's get some information about the target site from ZAP's API

#### Enumerating the Target Site

##### Information about the app's content - List of URLs

In [26]:
#fetch list of urls enumerated by ZAP
zap.core.urls()[:10]
#fetch upto 10 results (to fit on screen). You can remove the "[:10]" to fetch all results

['http://demo.testfire.net',
 'http://demo.testfire.net/admin',
 'http://demo.testfire.net/admin/clients.xls',
 'http://demo.testfire.net/admin',
 'http://demo.testfire.net/bank',
 'http://demo.testfire.net/bank/20060308_bak/',
 'http://demo.testfire.net/bank/account.aspx',
 'http://demo.testfire.net/bank/account.aspx.cs',
 'http://demo.testfire.net/bank/apply.aspx',
 'http://demo.testfire.net/bank/apply.aspx.cs']

Let's fetch some content types from the site with ZAP's API

In [29]:
display(zap.stats.site_stats(site = target_site)[:10])
#fetch upto 10 results (to fit on screen). You can remove the "[:10]" to fetch all results

[{'stats.code.200': 212, 'stats.code.301': 6, 'stats.code.302': 4, 'stats.code.401': 2, 'stats.code.403': 4, 'stats.code.404': 18, 'stats.code.500': 6, 'stats.contentType.application/javascript': 2, 'stats.contentType.application/octet-stream': 3, 'stats.contentType.application/pdf': 3, 'stats.contentType.application/rtf': 3, 'stats.contentType.application/vnd.ms-excel': 3, 'stats.contentType.image/gif': 15, 'stats.contentType.image/jpeg': 49, 'stats.contentType.text/css': 3, 'stats.contentType.text/html': 35, 'stats.contentType.text/html; charset=UTF-8': 7, 'stats.contentType.text/html; charset=utf-8': 124, 'stats.contentType.text/plain': 3, 'stats.contentType.text/xml': 4, 'stats.contentType.text/xml; charset=utf-8': 4, 'stats.responseTime.1024': 46, 'stats.responseTime.16384': 3, 'stats.responseTime.2048': 12, 'stats.responseTime.256': 24, 'stats.responseTime.4096': 4, 'stats.responseTime.512': 160, 'stats.responseTime.8192': 3, 'stats.tag.Comment': 42, 'stats.tag.Form': 122, 'stats

ZAP has a very useful list of params tested in the application. This can be accessed as follows....

In [44]:
zap.params.params()[0]['Parameter'][:5]
#fetch upto 5 results (to fit on screen). You can remove the "[0]['Parameter'][:5]" to fetch all results

[{'site': 'demo.testfire.net:80', 'name': 'amCreditOffer', 'timesUsed': '6', 'type': 'cookie'}, {'Flags': ['expires=Mon, 08-Jan-2018 09:37:04 GMT', 'path=/']}, {'Values': ['']}, {'site': 'demo.testfire.net:80', 'name': 'ASP.NET_SessionId', 'timesUsed': '243', 'type': 'cookie'}, {'Flags': ['session', 'path=/', 'HttpOnly']}]

Now, I am not sure if you noticed, but ZAP automatically starts gathering some details about vulnerabilities in the site using its `Passive Scan` Feature. This is very useful to silently enumerate vulnerabilites related to HTTP Headers, Session Tokens, etc without actually firing attack packets at the target.

#### Let's look at the stuf that ZAP Scans for passively...

In [30]:
display(zap.pscan.scanners[0:5])
#fetch upto 5 results (to fit on screen). You can remove the "[0:5]" to fetch all results

[{'alertThreshold': 'DEFAULT', 'name': 'Script Passive Scan Rules', 'id': '50001', 'enabled': 'true', 'quality': 'release'}, {'alertThreshold': 'DEFAULT', 'name': 'Stats Passive Scan Rule', 'id': '50003', 'enabled': 'true', 'quality': 'release'}, {'alertThreshold': 'DEFAULT', 'name': 'Application Error Disclosure', 'id': '90022', 'enabled': 'true', 'quality': 'release'}, {'alertThreshold': 'DEFAULT', 'name': 'Incomplete or No Cache-control and Pragma HTTP Header Set', 'id': '10015', 'enabled': 'true', 'quality': 'release'}, {'alertThreshold': 'DEFAULT', 'name': 'Content-Type Header Missing', 'id': '10019', 'enabled': 'true', 'quality': 'release'}]

As you can see from the above output, some of these are enabled and some are disabled. The number of findings from the Passive Scan are correlated with the number of URls that are crawled by ZAP. Before we go deep into vulnerability scanning, you can actually see the existing vulnerabilities that have been identified by ZAP with this...

In [33]:
# get an existing list of vulnerabilities
zap.core.alerts(baseurl=target_site)[:2]
#fetch upto 2 results (to fit on screen). You can remove the "[:2]" to fetch all results

[{'sourceid': '3', 'other': '', 'method': 'GET', 'evidence': 'Set-Cookie: amSessionId', 'pluginId': '10010', 'cweid': '16', 'confidence': 'Medium', 'wascid': '13', 'description': 'A cookie has been set without the HttpOnly flag, which means that the cookie can be accessed by JavaScript. If a malicious script can be run on this page then the cookie will be accessible and can be transmitted to another site. If this is a session cookie then session hijacking may be possible.', 'messageId': '1', 'url': 'http://demo.testfire.net/', 'reference': 'http://www.owasp.org/index.php/HttpOnly', 'solution': 'Ensure that the HttpOnly flag is set for all cookies.', 'alert': 'Cookie No HttpOnly Flag', 'param': 'amSessionId', 'attack': '', 'name': 'Cookie No HttpOnly Flag', 'risk': 'Low', 'id': '0'}, {'sourceid': '3', 'other': "The X-XSS-Protection HTTP response header allows the web server to enable or disable the web browser's XSS protection mechanism. The following values would attempt to enable it: 

#### Customizing Scan Policy

One of the things you might have noticed with DAST Scanning, is that with fully-enabled scan policies, it takes a significantly long time to perform the scan. Let's create a lightweight scan policy through ZAP's API. This is a new API endpoint in ZAP 2.7.0

In [34]:
#creating a new scan policy with attack threshold and attack strength
light = zap.ascan.add_scan_policy('Light', alertthreshold='Low', attackstrength='Low')

If everything has gone correctly, a new scan policy should have been created with the title "Light". Let's fetch all the scan policies in our ZAP instance

In [35]:
# querying all scan policies by name
zap.ascan.scan_policy_names

['Default Policy', 'Light']

## Let's start scanning!!

If you see "Light" the previous cell, it means that you are set and ready to scan. Let's get started....

In [36]:
# using ZAP's ascan object to start scanning, with the "Light" Policy. If you don't specify the policy
# ZAP Automatically uses the "Default" policy
active_scan_id = zap.ascan.scan(target_site, scanpolicyname='Light')

print("active scan id: {0}".format(active_scan_id))

#now we can start monitoring the spider's status
while int(zap.ascan.status(active_scan_id)) < 100:
    print("Current Status of ZAP Active Scan: {0}%".format(zap.ascan.status(active_scan_id)))
    time.sleep(10)

Current Status of ZAP Spider: 0%
Current Status of ZAP Spider: 0%
Current Status of ZAP Spider: 0%
Current Status of ZAP Spider: 0%
Current Status of ZAP Spider: 0%
Current Status of ZAP Spider: 0%
Current Status of ZAP Spider: 0%
Current Status of ZAP Spider: 0%
Current Status of ZAP Spider: 0%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 47%
Current Status of ZAP Spider: 48%
Current Status of ZAP Spider: 48%
Current Status of ZAP Spider: 48%
Current Status of ZAP Spider: 48%
Current Status of ZAP Spider: 48%
Current Status of ZAP S

### Pulling results to report

The next thing that we need after scanning is to pull results to generate some reports. ZAP provides an XML and HTML Report by default. However, with the "Export Report" plugin, ZAP allows you to export the XHTML, PDF and JSON variants of the report, replete with HTTP Requests, Responses and detailed information about the vulnerability. This can also be customized during the fetch, and done through the API

** Note: ZAP's Export Report functionality cannot be used through it's Python API. We will be using it's REST API for this API endpoint. Please refer to https://github.com/zaproxy/zap-extensions/wiki/HelpAddonsExportreportExportreport for additional details about ZAP's Export Report**

In [ ]:
url = 'http://localhost:8090/JSON/exportreport/action/generate/'
